# Shortest path calculations

We do simple shortest path calculations for the example network with a simple test.

The SP index are from u to v in order of the edge-list. This is most-likely due to the edge_list settings.

In [184]:
%reload_kedro
%load_ext autoreload
%autoreload 2
%config IPCompleter.use_jedi = False
import geopandas as gpd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import pandas as pd
import networkx as nx
import tables as tb
from mcarptif.converter.shortest_path import sp_full
from mcarptif.converter.network_prep import ShortestPathNodes

import logging
logging.basicConfig(level=logging.INFO)

2022-04-12 11:50:27,511 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-04-12 11:50:27,590 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-04-12 11:50:27,592 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-04-12 11:50:27,600 - root - INFO - Registered line magic `run_viz`
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load graph

In [185]:
node_list = catalog.load("road_network_simplified_24645_nodes_xy")
edge_list = catalog.load("road_network_simplified_24645_edges_xy")
edge_list["uv"] = edge_list["u"].astype(str) + "-" + edge_list["v"].astype(str)
edge_list = edge_list.sort_values(["u", "v", "key"])
node_list = node_list.set_index("index")
edge_list = edge_list.set_index(["u", "v", "key"])

G_sparse = ox.graph_from_gdfs(
    node_list,
    edge_list,
)

2022-04-12 11:50:28,304 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_nodes_xy` (GeoJSONDataSet)...
2022-04-12 11:50:28,716 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_edges_xy` (GeoJSONDataSet)...


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/osmnx/utils_graph.py:155: UserWarning: discarding the gdf_nodes geometry column, though its values differ from the coordinates in the x and y columns
  warnings.warn(


In [186]:
node_sp_index_order = pd.DataFrame({"osmid": list(G_sparse.nodes)})
node_sp_index_order["sp_index"] = node_sp_index_order.index
node_list = node_list.merge(node_sp_index_order, how="left", validate="1:1").sort_values("sp_index").reset_index(drop=True)

In [187]:
# file = "sp_files/full_road_simple_red.gpickle" # SG_full_road_simple_red
# logging.info(f'Reading OSMNX graph from `{file}`')
# G = nx.read_gpickle(file)
import os

if not os.path.isdir("../data/05_model_input/sp_files"):
    os.mkdir("../data/05_model_input/sp_files")

filename = "../data/05_model_input/sp_files/road_network_simplified_24645.h"

In [188]:
%time
sp_object = ShortestPathNodes(G_sparse, overwrite=True)
sp_object.calc_shortest_path_pytable(filename, f'SP calcs for: {filename}')

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs
2022-04-12 11:50:42,919 - root - WARNING - File ../data/05_model_input/sp_files/road_network_simplified_24645.h will be overwritten


Press 'y' to continue y


2022-04-12 11:50:46,278 - root - INFO - Generating 8145x8145 distance and predecessor matrix
2022-04-12 11:50:46,336 - root - INFO - Writing shortest path info to ../data/05_model_input/sp_files/road_network_simplified_24645.h
2022-04-12 11:50:46,380 - root - INFO - Starting shortest path calculations
2022-04-12 11:51:13,293 - root - INFO - Shortest path info added.
2022-04-12 11:51:13,294 - root - INFO - Shortest path calculations took 27.0 seconds
2022-04-12 11:51:13,296 - root - INFO - Done
2022-04-12 11:51:13,297 - root - INFO - h5file


## Test shortest-paths

In [194]:
filler_frames = []
arc_i = 164
arc_j = 165
with tb.open_file(filename, 'r') as h5file:
    sp_info = h5file.root.shortest_path_info
    p_full = sp_info.predecessor_matrix
    node_id_order = sp_info.node_id_order[:]
    sp_path = sp_full(p_full, arc_i, arc_j, full=True)

In [195]:
node_id_order

array([  21320956,   25525648,   25535600, ..., 9540385425, 9597340779,
       9597340792])

In [204]:
node_id_order[arc_i]

25535425

In [205]:
node_id_order[arc_j]

244003234

In [196]:
node_list.iloc[sp_path]

,osmid,y,x,highway,street_count,ref,geometry,sp_index
164,25535425,-33.951728,18.471958,None,3,None,POINT (-7480023.338 -9354800.479),164
165,244003234,-33.951411,18.471937,None,3,None,POINT (-7480085.515 -9354794.938),165


In [197]:
node_path = node_list.iloc[sp_path].copy()

In [198]:
node_path["u"] = node_path["osmid"].shift().values
node_path["v"] = node_path["osmid"]

In [199]:
node_path["uv"] = node_path["u"].fillna(-1).astype(int).astype(str) + "-" + node_path["v"].fillna(-1).astype(int).astype(str)

In [200]:
node_path["uv"].isin(edge_list["uv"])

164    False
165     True
Name: uv, dtype: bool

In [203]:
edge_list.loc[edge_list["uv"].isin(node_path["uv"])]

,,,osmid,lanes,name,highway,maxspeed,width,oneway,length,ref,access,bridge,tunnel,service,landuse,junction,geometry,uv
u,v,key,,,,,,,,,,,,,,,,,
25535425,244003234,0,152108216,2,Main Road,secondary,60,None,1,35.367,M4,None,None,None,None,None,None,"LINESTRING (-7480023.338 -9354800.479, -748008...",25535425-244003234
